LAB 2 - Milica miskovic

First, let us import all packages

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib as plt

ModuleNotFoundError: No module named 'matplotlib'

Then we read the data file into data frame

In [7]:
ratings = pd.read_csv('ratings.dat', sep="::", header=None, engine='python')
ratings.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings.drop(['Timestamp'], axis=1, inplace=True)
print(ratings[:5])

   UserID  MovieID  Rating
0       1     1193       5
1       1      661       3
2       1      914       3
3       1     3408       4
4       1     2355       5


1. Create m x u matrix with movies as row and users as column

In [160]:
ratings_pivot = ratings.pivot_table('Rating', index='MovieID', columns='UserID')
print(ratings_pivot.iloc[0:10, 0:10])

UserID    1   2   3   4    5    6    7    8    9    10
MovieID                                               
1        5.0 NaN NaN NaN  NaN  4.0  NaN  4.0  5.0  5.0
2        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  5.0
3        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
4        NaN NaN NaN NaN  NaN  NaN  NaN  3.0  NaN  NaN
5        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
6        NaN NaN NaN NaN  2.0  NaN  4.0  NaN  NaN  NaN
7        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  4.0
8        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
9        NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN
10       NaN NaN NaN NaN  NaN  NaN  NaN  NaN  NaN  NaN


2. Normalize the matrix (we have to fill NaNs with 0 first, and call it matrix a)

In [161]:
ratings_pivot.fillna(value=0, inplace=True)
a = StandardScaler().fit_transform(ratings_pivot)
print(a.shape)


(3706, 6040)


3. Compute SVD to get U, S and V

In [57]:
U, Sigma, V = np.linalg.svd(a)

4. From your V.T select 50 components (we have to reduce the other two matrices too, in order to SVD dim reduct)

In [87]:
U = U[:,:50]
S = np.diag(Sigma[0:50])
V = V[:50]


Shape of U ,S and V after reduction: 

In [88]:
print(U.shape)
print(S.shape)
print(V.shape)

(3706, 50)
(50, 50)
(50, 6040)


New, reduced matrix a we can recover by multiplying U ,S and V after reduction: 

In [143]:
a_reduced = U.dot(S).dot(V)
print(a_reduced.shape)

(3706, 6040)


In order to have movie names, we have to import movies.dat file.

In [123]:
movies = pd.read_csv('movies.dat', sep="::", header=None, engine='python')
movies.columns = ['MovieID', 'Title', 'Genres']
movies.drop(['Genres'], axis=1, inplace=True)
print(movies[:5])

   MovieID                               Title
0        1                    Toy Story (1995)
1        2                      Jumanji (1995)
2        3             Grumpier Old Men (1995)
3        4            Waiting to Exhale (1995)
4        5  Father of the Bride Part II (1995)


In [144]:
movies.loc[1, 'Title']

'Jumanji (1995)'

The function:

In [155]:
def recommend(movie_id):
    if movie_id > 0 and movie_id <= a_reduced.shape[0]:
        
        cos = np.zeros(a_reduced.shape[0])
        for i in range(0, a_reduced.shape[0]):
            a = np.dot(a_reduced[movie_id-1], a_reduced[i])
            b = np.linalg.norm(a_reduced[movie_id-1]) * np.linalg.norm(a_reduced[i])
            cos[i] = a/b
        order = np.argsort(cos)
        ten_most_similar = movies.loc[order[:10], 'Title']
    else:
        print('Invalid movie ID.')
    return(ten_most_similar)

Let us try with movie ID 2, 'Jumanji (1995)':

In [156]:
recommend(2)

[-0.29592026 -0.29409393 -0.29377709 ...  0.90664469  0.96011212
  1.        ]
[  51 2723  569 ...   58 3256    1]


51                                Mighty Aphrodite (1995)
2723                       Airplane II: The Sequel (1982)
569     Ciao, Professore! (Io speriamo che me la cavo ...
2491                                      Ravenous (1999)
1086                               American Strays (1996)
1359                                        Grease (1978)
392                                      Fall Time (1995)
1077                                    Doors, The (1991)
807                           Very Brady Sequel, A (1996)
1948                              Babes in Toyland (1961)
Name: Title, dtype: object

6. Repeat the same process except now instead of using SVD you will use PCA to get the eigenvectors.
7. You will require co-variance matrix as an input to your eig function.

Matrix a is already standardized and NaNs are filled with 0. 

Eigenvectors and eigenvalues:

In [ ]:
cov_mat = np.cov(a)

fig, ax = plt.subplots()
cax = ax.matshow(cov_mat)
fig.colorbar(cax, ticks=[0, 1])
plt.xticks(range(len(cov_mat.columns)), cov_mat.columns, rotation='vertical')
plt.yticks(range(len(cov_mat.columns)), cov_mat.columns)
plt.show()


NameError: name 'plt' is not defined

In [ ]:
eig_val, eig_vec = np.linalg.eig(cov_mat)
print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)